# HTTP Requests in Python (needs updates for Foundry)
This notebook runs the HTTP requests provided in the `test.http` file using Python.

In [ ]:
# Install required packages

!pip install requests azure.identity

In [ ]:
import requests
import json
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AccessToken

# Define variables
subscription_id = '00000000-0000-0000-0000-000000000000'
rg = 'dummy-rg'
capability_name = 'dummy-caphost'
hub_name = 'dummy-hub'
project_name = '=dummy-project'
vnet_name = 'dummy-vnet'
subnet_name = 'dummy-subnet'
openAiService = 'dummy-openai-service'
subnet_id = f'/subscriptions/{subscription_id}/resourceGroups/{rg}/providers/Microsoft.Network/virtualNetworks/{vnet_name}/subnets/{subnet_name}'
base_url = f'https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg}/providers/Microsoft.MachineLearningServices/workspaces/'

# Get token using DefaultAzureCredential
credential = DefaultAzureCredential()
token = credential.get_token("https://management.azure.com/.default")

# Define headers
headers = {
    'Authorization': f'Bearer {token.token}',
    'Content-Type': 'application/json'
}

# Define base URL
base_url = f'https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg}/providers/Microsoft.MachineLearningServices/workspaces/'


### Create Hub Capability Host

In [ ]:
url = f'{base_url}{hub_name}/capabilityHosts/{capability_name}?api-version=2024-10-01-preview'
payload = {
  "properties": {
    "capabilityHostKind": "Agents",
    "customerSubnet": subnet_id
  }
}
#print(json.dumps(payload, indent=4))
response = requests.put(url, headers=headers, data=json.dumps(payload))
print(json.dumps(response.json(), indent=4))

### Get Hub Capability Host

In [59]:
url = f'{base_url}{hub_name}/capabilityHosts/{capability_name}?api-version=2024-10-01-preview'
print(url)
response = requests.get(url, headers=headers)
print(json.dumps(response.json(), indent=4))

### Delete Hub Capability Host
Capability host needs to be deleted if it's in **Failed** state. Project capability needs to be deleted before hub capability.

In [ ]:
url = f'{base_url}{hub_name}/capabilityHosts/{capability_name}?api-version=2024-10-01-preview'
# avoid incidentally deleting the capability host
delete = False
if delete:
    response = requests.delete(url, headers=headers)
    print(json.dumps(response.json(), indent=4))

### Create Project Capability Host

In [ ]:
url = f'{base_url}{project_name}/capabilityHosts/{capability_name}?api-version=2024-10-01-preview'
payload = {
    "properties": {
        "capabilityHostKind": "Agents",
        "aiServicesConnections": [f"{hub_name}-connection-AIServices_aoai"],
        "vectorStoreConnections": [f"{hub_name}-connection-AISearch"],
        "storageConnections": [f"{project_name}/workspaceblobstore"]
    }
}
response = requests.put(url, headers=headers, data=json.dumps(payload))
print(json.dumps(response.json(), indent=4))

### Get Project Capability Host

In [60]:
url = f'{base_url}{project_name}/capabilityHosts/{capability_name}?api-version=2024-10-01-preview'
response = requests.get(url, headers=headers)
print(json.dumps(response.json(), indent=4))

### Delete Project Capability Host

In [ ]:
url = f'{base_url}{project_name}/capabilityHosts/{capability_name}?api-version=2024-10-01-preview'
# avoid incidentally deleting the capability host
delete = False
if delete:
    response = requests.delete(url, headers=headers)
    print(json.dumps(response.json(), indent=4))

### POST to Azure OpenAI Service

In [ ]:
ai_token = credential.get_token("https://cognitiveservices.azure.com/.default")
url = f'https://{openAiService}.openai.azure.com/openai/deployments/gpt-4o-mini/completions?api-version=2024-10-21'
headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + ai_token.token,
}
payload = {
    "messages": [
        {
            "role": "system",
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Does Azure OpenAI support customer managed keys?"
        },
        {
            "role": "assistant",
            "content": "Yes, customer managed keys are supported by Azure OpenAI."
        },
        {
            "role": "user",
            "content": "Do other Azure AI services support this too?"
        }
    ]
}
response = requests.post(url, headers=headers, data=json.dumps(payload))
print(json.dumps(response.json(), indent=4))